## 1. Graph Attention Networks

In [1]:
import torch
torchversion = torch.__version__

# Install PyTorch Scatter, PyTorch Sparse, and PyTorch Geometric
!pip install torch_geometric
!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-{torchversion}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-{torchversion}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

# Numpy for matrices
import numpy as np
np.random.seed(0)

# Visualization
import networkx as nx
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

### Dataset

In [2]:
from torch_geometric.datasets import Planetoid,KarateClub

# Import dataset from PyTorch Geometric
dataset = Planetoid(root=".", name="cora")

data = dataset[0]


C:\Users\user\anaconda3\envs\test\lib\site-packages\torch_geometric\typing.py:60: UserWarning: An issue occurred while importing 'torch-scatter'. Disabling its usage. Stacktrace: Could not find module 'C:\Users\user\anaconda3\envs\test\Lib\site-packages\torch_scatter\_scatter_cuda.pyd' (or one of its dependencies). Try using the full path with constructor syntax.
  warnings.warn(f"An issue occurred while importing 'torch-scatter'. "
C:\Users\user\anaconda3\envs\test\lib\site-packages\torch_geometric\typing.py:98: UserWarning: An issue occurred while importing 'torch-sparse'. Disabling its usage. Stacktrace: Could not find module 'C:\Users\user\anaconda3\envs\test\Lib\site-packages\torch_sparse\_convert_cuda.pyd' (or one of its dependencies). Try using the full path with constructor syntax.
  warnings.warn(f"An issue occurred while importing 'torch-sparse'. "
Processing...
Done!


### Print information about the dataset

In [3]:
# Print information about the dataset
print(f'Dataset: {dataset}')
print('-------------------')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of nodes: {data.x.shape[0]}')
print(f'Number of features: {dataset.num_features}')
print(f'Number of classes: {dataset.num_classes}')

# Print information about the graph
print(f'\nGraph:')
print('------')
print(f'Edges are directed: {data.is_directed()}')
print(f'Graph has isolated nodes: {data.has_isolated_nodes()}')
print(f'Graph has loops: {data.has_self_loops()}')

Dataset: cora()
-------------------
Number of graphs: 1
Number of nodes: 2708
Number of features: 1433
Number of classes: 7

Graph:
------
Edges are directed: False
Graph has isolated nodes: False
Graph has loops: False


In [4]:
from torch_geometric.utils import remove_isolated_nodes

isolated = (remove_isolated_nodes(data['edge_index'])[2] == False).sum(dim=0).item()
print(f'Number of isolated nodes = {isolated}')

Number of isolated nodes = 0


### Implement GAT

#### GAT model

In [10]:
class GAT(torch.nn.Module):
  """Graph Attention Network"""
  def __init__(self, dim_in, dim_h, dim_out, heads=8):
    super().__init__()
    self.gat1 = GATv2Conv(dim_in, dim_h, heads=heads)
    self.gat2 = GATv2Conv(dim_h*heads, dim_out, heads=1)
    self.optimizer = torch.optim.Adam(self.parameters(),
                                      lr=0.005,
                                      weight_decay=5e-4)

  def forward(self, x, edge_index):
    h = F.dropout(x, p=0.6, training=self.training)
    h = self.gat1(x, edge_index)
    h = F.elu(h)
    h = F.dropout(h, p=0.6, training=self.training)
    h = self.gat2(h, edge_index)
    return h, F.log_softmax(h, dim=1)

### Train function

In [11]:
def accuracy(pred_y, y):
    """Calculate accuracy."""
    return ((pred_y == y).sum() / len(y)).item()

def train(model, data):
    """Train a GNN model and return the trained model."""
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = model.optimizer
    epochs = 5

    model.train()
    for epoch in range(epochs+1):
        # Training
        optimizer.zero_grad()
        _, out = model(data.x, data.edge_index)
        loss = criterion(out[data.train_mask], data.y[data.train_mask])
        acc = accuracy(out[data.train_mask].argmax(dim=1), data.y[data.train_mask])
        loss.backward()
        optimizer.step()

        # Validation
        val_loss = criterion(out[data.val_mask], data.y[data.val_mask])
        val_acc = accuracy(out[data.val_mask].argmax(dim=1), data.y[data.val_mask])

        # Print metrics every 10 epochs
        if(epoch % 1 == 0):
            print(f'Epoch {epoch:>3} | Train Loss: {loss:.3f} | Train Acc: '
                  f'{acc*100:>6.2f}% | Val Loss: {val_loss:.2f} | '
                  f'Val Acc: {val_acc*100:.2f}%')

    return model

In [12]:
def test(model, data):
    """Evaluate the model on test set and print the accuracy score."""
    model.eval()
    _, out = model(data.x, data.edge_index)
    acc = accuracy(out.argmax(dim=1)[data.test_mask], data.y[data.test_mask])
    return acc

### Train GAT

In [13]:
%%time

# Create GAT model
gat = GAT(dataset.num_features, 8, dataset.num_classes)
print(gat)

GAT(
  (gat1): GATv2Conv(1433, 8, heads=8)
  (gat2): GATv2Conv(64, 7, heads=1)
)
CPU times: total: 31.2 ms
Wall time: 16 ms


In [15]:


# Train
train(gat, data)

# Test
acc = test(gat, data)
print(f'\nGAT test accuracy: {acc*100:.2f}%\n')

Epoch   0 | Train Loss: 0.797 | Train Acc:  97.14% | Val Loss: 1.29 | Val Acc: 75.80%
Epoch   1 | Train Loss: 0.679 | Train Acc:  98.57% | Val Loss: 1.19 | Val Acc: 75.40%
Epoch   2 | Train Loss: 0.560 | Train Acc:  98.57% | Val Loss: 1.12 | Val Acc: 76.00%
Epoch   3 | Train Loss: 0.465 | Train Acc:  97.14% | Val Loss: 1.07 | Val Acc: 75.40%
Epoch   4 | Train Loss: 0.396 | Train Acc:  97.86% | Val Loss: 1.01 | Val Acc: 74.80%
Epoch   5 | Train Loss: 0.322 | Train Acc:  97.14% | Val Loss: 0.99 | Val Acc: 74.60%

GAT test accuracy: 77.90%



### GATv2

In [17]:
class GATv2(torch.nn.Module):
    def __init__(self,num_features,dims, num_classes,  drop=0.0):
        super(GATv2, self).__init__()
        heads = 8
        self.h = None
        self.conv1 = GATv2Conv(num_features,dims, heads=heads, dropout = 0.3, concat=False)
        self.conv2 = GATv2Conv(dims, num_classes, heads=heads, concat=False, dropout=0.3)
        self.drop = torch.nn.Dropout(p=drop)
        self.optimizer = torch.optim.Adam(self.parameters(),lr=0.005,weight_decay=5e-4)
    def forward(self, x, edge_index,):
        x = F.elu(self.conv1(x, edge_index))
        x = self.drop(x)
        x = self.conv2(x, edge_index)
        return x, F.log_softmax(x, dim=1)

In [18]:
%%time

# Create GAT model
gat2 = GATv2(dataset.num_features, 8, dataset.num_classes)
print(gat2)

GATv2(
  (conv1): GATv2Conv(1433, 7, heads=8)
  (conv2): GATv2Conv(7, 8, heads=8)
  (drop): Dropout(p=0.0, inplace=False)
)
CPU times: total: 15.6 ms
Wall time: 4 ms


In [19]:
# Train
train(gat2, data)

# Test
acc = test(gat2, data)
print(f'\nGAT test accuracy: {acc*100:.2f}%\n')

Epoch   0 | Train Loss: 2.085 | Train Acc:  12.86% | Val Loss: 2.05 | Val Acc: 15.20%
Epoch   1 | Train Loss: 2.069 | Train Acc:  19.29% | Val Loss: 2.04 | Val Acc: 19.80%
Epoch   2 | Train Loss: 2.054 | Train Acc:  27.14% | Val Loss: 2.03 | Val Acc: 28.80%
Epoch   3 | Train Loss: 2.037 | Train Acc:  31.43% | Val Loss: 2.02 | Val Acc: 33.20%
Epoch   4 | Train Loss: 2.021 | Train Acc:  35.00% | Val Loss: 2.01 | Val Acc: 38.40%
Epoch   5 | Train Loss: 1.997 | Train Acc:  38.57% | Val Loss: 1.99 | Val Acc: 40.40%

GAT test accuracy: 46.80%

